In [10]:
import json
import glob
import pandas as pd
import os
import re

In [11]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)   # URL sil
    text = re.sub(r"@\w+", "", text)      # Mention sil
    text = re.sub(r"#\w+", "", text)      # Hashtag sil
    text = re.sub(r"\s+", " ", text).strip()   # Fazla boşluk
    return text

In [12]:
import json
import glob
import pandas as pd

# Tüm alanları tek satıra topla
all_rows = []
# Tüm yılları ve günleri bulmak için recursive glob kullan!
for file_path in glob.glob("front_page_news_json_format/**/*.json", recursive=True):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    # Eğer dosyada metadata varsa tarih al, yoksa filename'den çek
    meta_date = data.get("metadata", {}).get("date", os.path.splitext(os.path.basename(file_path))[0])
    for article in data["articles"]:
        headline_clean = preprocess_text(article.get("headline", ""))
        abstract_clean = preprocess_text(article.get("abstract", ""))
        lead_paragraph_clean = preprocess_text(article.get("lead_paragraph", ""))
        # Hepsini birleştir: fulltext
        fulltext_clean = " ".join([headline_clean, abstract_clean, lead_paragraph_clean]).strip()
        row = {
            "date": meta_date,
            "fulltext_clean": fulltext_clean,
            "section_name": article.get("section_name", ""),
        }
        all_rows.append(row)

df = pd.DataFrame(all_rows)
df.to_csv("nyt_frontpage_all_clean.csv", index=False)
print("Tüm haberler temizlenmiş şekilde tek bir CSV’ye kaydedildi.")

Tüm haberler temizlenmiş şekilde tek bir CSV’ye kaydedildi.
